In [148]:
import torch
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, TensorDataset
from torch.optim import SGD
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import torchmetrics
import torchvision
from torchvision.transforms import Compose, ToTensor
from torch import flatten

IMG_SIZE = 250

In [149]:
    
def Get_data_old():
    """This will return X and y train and test sets of the lfw people data set
        Specifically returns:
        y_train, y_test and X_train, X_test"""
    
    #Download the data, if not already on disk and load it as numpy arrays
    lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4, data_home="./images")

    # The following code could be used for data preprocessing if using pytorch
    X = lfw_people.images
    Y = lfw_people.target

    # Verify the value range of X_train. No normalization is necessary in this case,
    # as the input values already fall within the range of 0.0 to 1.0.
    print("X_min:",X.min(),"X_train_max:", X.max())

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
    X_train = X_train[:, np.newaxis, :, :]
    X_test = X_test[:, np.newaxis, :, :]
    
    print("X_train shape:", X_train.shape)

    #Convert to tensors 
    X_train = torch.from_numpy(X_train) #maybe .to(torch.float32)
    X_test = torch.from_numpy(X_test)
    y_train = torch.from_numpy(y_train)
    y_test = torch.from_numpy(y_test)

    dic_data = {}
    dic_data['train'] = TensorDataset(X_train, y_train)
    dic_data['test'] = TensorDataset(X_test, y_test)

    print("Data Loaded")

    return dic_data

In [150]:

class lfw_mod(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.fc1 = nn.Linear(2464, 500)
        self.fc2 = nn.Linear(500, 7)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()
        self.mxpool1 = nn.MaxPool2d(2)
        self.mxpool2 = nn.MaxPool2d(2)
        self.softmax = nn.LogSoftmax(dim=1)
        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=7)

        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.mxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.mxpool2(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)

        x = self.fc2(x)
        output = self.softmax(x)
        return output
    
    def configure_optimizers(self):
        LR = 1e-3
        optimizer = torch.optim.AdamW(self.parameters(),lr=LR)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x,y = batch
        out = self(x)
        loss = F.cross_entropy(out,y)
        self.log('train_loss', loss,on_step=True,on_epoch=True)
        return loss 

    def test_step(self,batch,batch_idx):
        x,y = batch
        out = self(x)
        loss = F.cross_entropy(out,y)
        out = nn.Softmax(-1)(out) 
        logits = torch.argmax(out,dim=1)
        accu = self.accuracy(logits, y)        
        self.log('test_loss', loss)
        self.log('train_acc_step', accu)
        return loss, accu
    
    def validation_step(self,batch,batch_idx):
        x,y = batch
        out = self(x)
        loss = F.cross_entropy(out,y)
        out = nn.Softmax(-1)(out) 
        logits = torch.argmax(out,dim=1)
        accu = self.accuracy(logits, y)        
        self.log('Val_loss', loss)
        self.log('Val_acc_step', accu)
        return loss, accu
    
    
    
    


In [151]:
def get_data():
    dataset = torchvision.datasets.ImageFolder(
        'C:\Data\COMP3710\Pattern-Recognition\images\lfw_home\lfw_funneled', 
        transform= Compose([ToTensor()]))
    train_idx, test_idx = train_test_split(list(range(len(dataset))),test_size=0.25, random_state=42)
    print(len(dataset))
    dic_data = {}
    dic_data['train'] = Subset(dataset, train_idx)
    dic_data['test'] = Subset(dataset, test_idx)
    print(len(dic_data['train']))
    print(dic_data['test'])
    return dic_data

In [157]:
class load_data(pl.LightningDataModule):
    def __init__(self) -> None:
        super().__init__()
        self.batch_size = 11
        dataset = Get_data_old()
        self.train = dataset['train']
        self.test = dataset['test']

    def train_dataloader(self):
        return DataLoader(self.train, self.batch_size)    

    def test_dataloader(self):
        return DataLoader(self.test, self.batch_size)
    
    def val_dataloader(self):
        return DataLoader(self.test, self.batch_size)
        


In [158]:
def main():
     mod = lfw_mod()
     data = load_data()
     trainer = pl.Trainer(max_epochs=10)
     trainer.fit(mod, data)
     trainer.test(mod, data)
     

if __name__ == '__main__': main()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name     | Type               | Params
-------------------------------------------------
0  | conv1    | Conv2d             | 320   
1  | conv2    | Conv2d             | 9.2 K 
2  | fc1      | Linear             | 1.2 M 
3  | fc2      | Linear             | 3.5 K 
4  | relu1    | ReLU               | 0     
5  | relu2    | ReLU               | 0     
6  | relu3    | ReLU               | 0     
7  | mxpool1  | MaxPool2d          | 0     
8  | mxpool2  | MaxPool2d          | 0     
9  | softmax  | LogSoftmax         | 0     
10 | accuracy | MulticlassAccuracy | 0     
-------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.982     Total estimated model params size (MB)


X_min: 0.0 X_train_max: 1.0
X_train shape: (966, 1, 50, 37)
Data Loaded
Epoch 9: 100%|██████████| 88/88 [00:02<00:00, 31.13it/s, v_num=50]          

`Trainer.fit` stopped: `max_epochs=10` reached.


Testing DataLoader 0: 100%|██████████| 30/30 [00:00<00:00, 113.27it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.6081709861755371
     train_acc_step         0.8322981595993042
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
